<font color='black'>
<h1>
<span style="font-family:verdana; font-size:1.4em;">
Writing Data
</span>
</h1>
</font>

<font color='steelblue'>
<h2>
<span style="font-family:verdana; font-size:1.4em;">
Examples & Exercise on Data Partitioning<br>
</span>
</h2>
</font>

<font color='grey'>
<span style="font-family:verdana; font-size:1.2em;">

Writing data in Spark is different compared to the traditional databases due to the distributed nature of Spark data.<br>

</span>
<span style="font-family:verdana; font-size:1.0em;"><br>
    <b>Following is included in this notebook</b>:
    <ul>
        <li>Write dataframe with different number of partitions</li>
        <li>Using Spark UI</li>
        <li><b>Exercise 1:</b> Changing the number of partitions and examining the record count for each partition</li>
    </ul>
</span>
</font>

<font color='tomato'>
<span style="font-family:verdana; font-size:1.2em;">
    <b>IF RUNNING IN DATABRICK environment, set the next cell to True</b>:
</span>
</font>

In [ ]:
DATA_BRICKS = False

In [ ]:
if DATA_BRICKS == False:
    # Set up the environment for using pyspark
    import findspark
    findspark.init()
    
    from pyspark.sql import SparkSession
    from pyspark import SparkConf, SparkContext
    
    spark = SparkSession.builder.appName("Data Writes").getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("Error")

In [ ]:
# Data type definitions, # Lot of functions used in this notebook
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Utility Functions

In [ ]:
def showDataFrame(df, numrecs = 5):
    if DATA_BRICKS == False:
        df.limit(numrecs).show(truncate = False)
    else:
        display(df)

def listFiles(dirname):
    if DATA_BRICKS == False:
      import os
      ret = os.listdir(dirname)
      for f in ret:
        print(f)
    else:
      ret = dbutils.fs.ls("dbfs:/" + dirname)
      print("Number of files {} Filenames are:".format(len(ret)))
      for f in ret:
        print(f.name)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Load wiki data<br>
</span>
</h3>
</font>

In [ ]:
if DATA_BRICKS == False:
    path = "../datasets/"
else:
    path = "dbfs:/FileStore/datasets/"

In [ ]:
csvfile = path + "wikidata.csv"

In [ ]:
wSchema = StructType([
    StructField("timestamp", StringType(), True), 
    StructField("site", StringType(), True),
    StructField("requests", IntegerType(), True),
    StructField("dow", StringType(), True)
])

In [ ]:
from pyspark.sql.functions import col, date_format

wikiDF = spark.read.csv(csvfile, header = True, schema = wSchema)

In [ ]:
showDataFrame(wikiDF)

In [ ]:
partitions = wikiDF.rdd.getNumPartitions()
print("Partitions: {0:,}".format(partitions))

### Increase the number of partitions to 16

In [ ]:
repartDF = wikiDF.repartition(16)
print("Partitions: {0:,}".format(repartDF.rdd.getNumPartitions()))

### Reduce the number of partitions

In [ ]:
reducedDF = repartDF.coalesce(2)
print("Partitions: {0:,}".format(reducedDF.rdd.getNumPartitions()))

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Set default partitions<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
The default number of partitions is 200, which makes sense when dealing with large distributed datasets.<br>
The following shows example of reconfiguring this default value
</span>

In [ ]:
spark.conf.get("spark.sql.shuffle.partitions")

### Set the default number of partitions to 8

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "8")

<span style="font-family:verdana; font-size:1.0em;">
    Now let us use the reducedDF (coalesced it to 8 partitions) above. Apply <i>orderBy</i> to see how the changes to the default number of partitions impact the operation
<span>

In [ ]:
orderedParts = reducedDF.orderBy("requests").rdd.getNumPartitions()
print("Paritions: {0:,}".format(orderedParts))

### Reset the default value

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "200")

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Parallel Writes<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
The default number of partitions is 200, which makes sense when dealing with large distributed datasets.<br>
The following shows example of reconfiguring this default value
</span>

In [ ]:
if DATA_BRICKS == False:
    workingDir = "../output/"
else:
    workingDir = "/FileStore/output/"
targetPath = workingDir + "wikiDataProcessed"
print(targetPath)

In [ ]:
print(wikiDF.rdd.getNumPartitions())

In [ ]:
wikiDF = wikiDF.coalesce(5)

In [ ]:
print(wikiDF.rdd.getNumPartitions())

In [ ]:
# write the wikiDF as parquet 
wikiDF.write.mode("overwrite").parquet(targetPath)

In [ ]:
listFiles(targetPath)

#### There are 5 partitions indicating that Spark used 5 different connections to this directory

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Examine the Spark UI<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
    <b>On Databrick platform</b> Click the arrow next to <i>Spark Jobs</i> to see the breakdown of the stages while writing the data.<br>
    <b>Running on VM or local machine</b> Use localhost:4040 or localhost:4041 to observe the same information<br>
</span>

In [ ]:
wikiDF.repartition(12).write.mode("OVERWRITE").parquet(targetPath)

In [ ]:
listFiles(targetPath)

In [ ]:
wikiDF.repartition(10).write.mode("OVERWRITE").parquet(targetPath)

In [ ]:
listFiles(targetPath)

<font color='tomato'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Exercise 1: Changing Number of Partitions<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Will use the wikiDF dataframe that we defined above and then repartition it into different partitions<br>
Perform the following steps:
    <ol>
        <li>Use the utility function defined below on wikiDF</li>
        <li>Repartition wikiDF into 3 different dataframes with partitions 1, 16 and 64</li>
        <li>Use the utility function to print record count for these new dataframes</li>
        <li>Drop the number of partitions on dataframe with 64 partition to 8 partitions</li>
        <li>Use utility function to print counts on this repartitioned dataframe</li>
    </ol>
</span>

### Utility Function to print number of records per partition

In [ ]:
def printRecordsPerPartition(df):
    '''
    Utility method to count & print the number of records in each partition
    '''
    print("Per-Partition Counts:")
    def countInPartition(iterator): 
        yield __builtin__.sum(1 for _ in iterator)
        
    results = (df.rdd                   # Convert to an RDD
               .mapPartitions(countInPartition)  # For each partition, count
               .collect()                        # Return the counts to the driver
              )
        
    for result in results: 
        print("* " + str(result))

### Use the function defined above on the wikiDF dataframe

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Repartition the wikiDF<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Reparition the dataframe into 3 new dataframes:<br>
    <ul>
        <li><i>wikiDF1Partition: </i> wikiDF with 1 partition</li>
        <li><i>wikiDF16Partition: </i> wikiDF with 16 partition</li>
        <li><i>wikiDF64Partition: </i> wikiDF with 64 partition</li>
    </ul>
</span>

### Print distribution record count using the utility function defined above

### Coalesce <i>wikiDF64Partition</i> to 8 partitions

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Resources<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
<a href="https://databricks.com/blog/2015/06/22/understanding-your-spark-application-through-visualization.html" target="_blank">Understanding your Spark Application through Visualization</a><br>
<a href="https://spark.apache.org/docs/3.0.0-preview/web-ui.html" target="_blank">Spark UI 3.0 documentation</a>
</span>